In [1]:
from api_token import token
import todoist as td
import pandas as pd
import datetime as dt

In [2]:
api = td.TodoistAPI(token)
# Sync to API to get latest data
api.sync();

In [3]:
# when the api query has no items (tasks) to report, it returns a dictionary with two empty keys e.g.
# {'items':[], 'projects':[]}
# Therefore I need to create a function to check if the keys in the dictionary have values

def has_values(dictionary):
    
    # For each key in dictionary, check if keys in dict have values
    values = [any(dictionary[key]) for key in dictionary]
    
    # Create a unique list of values
    output = list(set(values))
    
    return output[0]

In [4]:
# The API has a maximum of 200 items that can be returned, use while loop to get all items

completed_tasks_list = []
limit = 200
offset = 0
not_empty = True

while not_empty:
    # API call, retrieving between 0 and 200 items at a time
    completed_tasks = api.completed.get_all(limit=limit, offset=offset)
    not_empty = has_values(completed_tasks) # if false it will exit the loop
    completed_tasks_list.append(completed_tasks) # add items to list
    offset += limit # increase offset to get the next set of results
    

In [5]:
# Create a list of dfs for each result of items
completed_dfs = [pd.DataFrame(completed_tasks_list[i]['items']) for i in range(0,len(completed_tasks_list))]

In [6]:
# Combine dfs into one large df
df = pd.concat(completed_dfs, ignore_index = True)

In [7]:
# Check num of rows and columns
df.shape

(1004, 7)

In [8]:
# Our df doesn't yet have project names, only project ids, we want to add a field with project names
td_projects_dict = {}
for i,data in enumerate(completed_tasks_list):
    for key in data['projects'].keys():
        td_projects_dict[int(key)] = data['projects'][key]['name']

In [9]:
# Map project names to df
df['project_name'] = df['project_id'].map(td_projects_dict)

In [10]:
# TODO Create statement that prints the following:
# You completed XXX tasks in 2020. That’s an average of…
# XX tasks per day
# XX tasks per week
# XX tasks per month

In [11]:
# TODO first add a column that will get the year of completion

# Create func to convert time to datetime object
def convert_dt_object(date):
    date_format = "%Y-%m-%dT%H:%M:%SZ"
    output = dt.datetime.strptime(date, date_format)
    return(output)

In [12]:
# Convert date to datetime object
df['completed_datetime_object'] = df['completed_date'].map(convert_dt_object)

In [18]:
df['completed_date_year'] = df['completed_datetime_object'].map(lambda x: x.year)
df['completed_date_month'] = df['completed_datetime_object'].map(lambda x: x.month)
df['completed_date_weekday'] = df['completed_datetime_object'].map(lambda x: x.weekday())

In [15]:
# Get count of tasks completed in each year
df.groupby("completed_date_year", as_index=False)["task_id"].count()

,completed_date_year,task_id
0,2020,837
1,2021,167


In [ ]:
completed_tasks_2020 = df.groupby("completed_date_year")["task_id"].count()[2020]

In [ ]:
print(f"""You completed {completed_tasks_2020} tasks in 2020. That's an average of...
{int(round(completed_tasks_2020/365,0))} tasks per day
{int(round(completed_tasks_2020/52,0))} tasks per week
{int(round(completed_tasks_2020/12,0))} tasks per month""")

In [19]:
df.head()

,completed_date,content,id,meta_data,project_id,task_id,user_id,project_name,completed_datetime_object,completed_date_year,completed_date_month,completed_date_weekday
0,2021-01-19T14:10:07Z,Enquire about life insurance,4210159397,None,2240338162,4476148360,28966387,🥓 JET,2021-01-19 14:10:07,2021,1,1
1,2021-01-19T10:17:34Z,**00** Daily planning,4209670308,None,2240338076,4476210762,28966387,❶ This Week,2021-01-19 10:17:34,2021,1,1
2,2021-01-19T10:13:49Z,Baby potatoes,4209663482,None,2240338076,4499030537,28966387,❶ This Week,2021-01-19 10:13:49,2021,1,1
3,2021-01-19T10:13:45Z,Millys bday,4209663366,None,2250738735,4478505218,28966387,❷ Next Week,2021-01-19 10:13:45,2021,1,1
4,2021-01-18T20:39:00Z,Call Barnes and Partners to make payment via ...,4208442482,None,2240338076,4481178181,28966387,❶ This Week,2021-01-18 20:39:00,2021,1,0


In [20]:
df.groupby(["completed_date_year", "completed_date_month"], as_index=False)["task_id"].count()

,completed_date_year,completed_date_month,task_id
0,2020,7,123
1,2020,8,184
2,2020,9,79
3,2020,10,23
4,2020,11,172
5,2020,12,256
6,2021,1,167


In [21]:
df.groupby(["completed_date_year", "completed_date_weekday"], as_index=False)["task_id"].count()

,completed_date_year,completed_date_weekday,task_id
0,2020,0,164
1,2020,1,114
2,2020,2,130
3,2020,3,78
4,2020,4,108
5,2020,5,138
6,2020,6,105
7,2021,0,34
8,2021,1,20
9,2021,2,13
